Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [5]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [7]:
pickle_file = 'models/notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [8]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [9]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

### Logistic Regression

In [10]:
batch_size = 128
beta = 0.05

graph = tf.Graph()
with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_train_labels, logits=logits)) +( 
        beta * tf.nn.l2_loss(weights))

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.001).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)
    

In [11]:
num_steps = 100000

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]

        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 166.709671
Minibatch accuracy: 10.9%
Validation accuracy: 8.4%
Minibatch loss at step 500: 157.664139
Minibatch accuracy: 12.5%
Validation accuracy: 10.9%
Minibatch loss at step 1000: 148.989197
Minibatch accuracy: 14.8%
Validation accuracy: 15.2%
Minibatch loss at step 1500: 138.861649
Minibatch accuracy: 18.8%
Validation accuracy: 18.9%
Minibatch loss at step 2000: 131.965942
Minibatch accuracy: 21.9%
Validation accuracy: 23.1%
Minibatch loss at step 2500: 124.987946
Minibatch accuracy: 21.9%
Validation accuracy: 26.4%
Minibatch loss at step 3000: 118.347038
Minibatch accuracy: 26.6%
Validation accuracy: 30.3%
Minibatch loss at step 3500: 111.906174
Minibatch accuracy: 31.2%
Validation accuracy: 33.6%
Minibatch loss at step 4000: 105.512085
Minibatch accuracy: 40.6%
Validation accuracy: 36.5%
Minibatch loss at step 4500: 100.136276
Minibatch accuracy: 43.8%
Validation accuracy: 39.5%
Minibatch loss at step 5000: 94.598366
Minibatch accuracy: 43.0

Minibatch loss at step 44500: 2.615086
Minibatch accuracy: 78.1%
Validation accuracy: 80.1%
Minibatch loss at step 45000: 2.558762
Minibatch accuracy: 77.3%
Validation accuracy: 80.0%
Minibatch loss at step 45500: 2.288686
Minibatch accuracy: 83.6%
Validation accuracy: 80.1%
Minibatch loss at step 46000: 2.160003
Minibatch accuracy: 82.8%
Validation accuracy: 80.2%
Minibatch loss at step 46500: 2.442974
Minibatch accuracy: 70.3%
Validation accuracy: 80.2%
Minibatch loss at step 47000: 2.110536
Minibatch accuracy: 84.4%
Validation accuracy: 80.3%
Minibatch loss at step 47500: 2.025946
Minibatch accuracy: 83.6%
Validation accuracy: 80.4%
Minibatch loss at step 48000: 1.871253
Minibatch accuracy: 87.5%
Validation accuracy: 80.4%
Minibatch loss at step 48500: 1.991657
Minibatch accuracy: 80.5%
Validation accuracy: 80.4%
Minibatch loss at step 49000: 1.981275
Minibatch accuracy: 78.9%
Validation accuracy: 80.6%
Minibatch loss at step 49500: 1.970951
Minibatch accuracy: 80.5%
Validation accu

Minibatch loss at step 89500: 0.889096
Minibatch accuracy: 82.8%
Validation accuracy: 81.4%
Minibatch loss at step 90000: 0.849699
Minibatch accuracy: 86.7%
Validation accuracy: 81.4%
Minibatch loss at step 90500: 0.799182
Minibatch accuracy: 85.9%
Validation accuracy: 81.3%
Minibatch loss at step 91000: 1.077206
Minibatch accuracy: 73.4%
Validation accuracy: 81.4%
Minibatch loss at step 91500: 0.918901
Minibatch accuracy: 78.9%
Validation accuracy: 81.4%
Minibatch loss at step 92000: 0.840500
Minibatch accuracy: 82.8%
Validation accuracy: 81.4%
Minibatch loss at step 92500: 0.800279
Minibatch accuracy: 86.7%
Validation accuracy: 81.4%
Minibatch loss at step 93000: 0.911803
Minibatch accuracy: 78.1%
Validation accuracy: 81.4%
Minibatch loss at step 93500: 0.853450
Minibatch accuracy: 79.7%
Validation accuracy: 81.4%
Minibatch loss at step 94000: 0.904203
Minibatch accuracy: 82.0%
Validation accuracy: 81.4%
Minibatch loss at step 94500: 0.916424
Minibatch accuracy: 81.2%
Validation accu

### Neural Network (1024 hidden nodes) with regularization

In [65]:
batch_size = 64
hidden_nodes_1 = 3072
hidden_nodes_2 = 1024
beta = 0.00
num_labels = 10

# x (-1, 784)
# W1 (784, 1024)
# biases1 (1024, 1)
# relu (el output seria el nuevo x) (-1, 1024)
# W2 (1024, 10)
# biases2 (10, 1)
# termina con el softmax_cross_entropy

graph = tf.Graph()
with graph.as_default():
    keep_prob = tf.placeholder(tf.float32)
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    weights = {
        'w1': tf.Variable(tf.random_normal([image_size * image_size, hidden_nodes_1], stddev=0.1)),
        'w2': tf.Variable(tf.random_normal([hidden_nodes_1, hidden_nodes_2], stddev=0.1)),
        'out': tf.Variable(tf.random_normal([hidden_nodes_2, num_labels], stddev=0.1))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([hidden_nodes_1], stddev=0.1)),
        'b2': tf.Variable(tf.random_normal([hidden_nodes_2], stddev=0.1)),
        'out': tf.Variable(tf.random_normal([num_labels], stddev=0.1))
    }

    layer_1 =  tf.nn.dropout(tf.nn.relu((tf.add(tf.matmul(tf_train_dataset, weights['w1']), biases['b1']))), keep_prob)
    layer_2 =  tf.nn.dropout(tf.nn.relu((tf.add(tf.matmul(layer_1, weights['w2']), biases['b2']))), keep_prob)
    out_layer = tf.add(tf.matmul(layer_2, weights['out']), biases['out'])
    
    logits = out_layer
    

    loss = tf.reduce_mean(
         tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_train_labels, logits=logits)) + ((
         beta * tf.nn.l2_loss(layer_1)) + (
         beta * tf.nn.l2_loss(layer_2)) #+ (
         #beta * tf.nn.l2_loss(out_layer))
    )
    
    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.00001, global_step, decay_steps =100, decay_rate=0.99)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    #optimizer = tf.train.GradientDescentOptimizer(0.001).minimize(loss)

    train_prediction = tf.nn.softmax(logits)
    
    valid_layer_1 =tf.nn.relu(tf.add(tf.matmul(tf_valid_dataset, weights['w1']), biases['b1']))
    valid_layer_2 = tf.nn.relu(tf.add(tf.matmul(valid_layer_1, weights['w2']), biases['b2']))
    valid_out_layer = tf.add(tf.matmul(valid_layer_2, weights['out']), biases['out'])
    valid_prediction = tf.nn.softmax(valid_out_layer)
    
    test_layer_1 =  tf.nn.relu(tf.add(tf.matmul(tf_test_dataset, weights['w1']), biases['b1']))
    test_layer_2 = tf.nn.relu(tf.add(tf.matmul(test_layer_1, weights['w2']), biases['b2']))
    test_out_layer = tf.add(tf.matmul(test_layer_2, weights['out']), biases['out'])
    test_prediction = tf.nn.softmax(test_out_layer)
    
    
    

    


In [66]:
num_steps = 40001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)

        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels,  keep_prob: 0.99}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 1000 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
            print("Learning Rate: {0}".format(learning_rate.eval()))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 17.420126
Minibatch accuracy: 15.6%
Validation accuracy: 9.5%
Learning Rate: 9.99899384623859e-06
Minibatch loss at step 1000: 8.766129
Minibatch accuracy: 20.3%
Validation accuracy: 18.6%
Learning Rate: 9.042912097356748e-06
Minibatch loss at step 2000: 5.054042
Minibatch accuracy: 31.2%
Validation accuracy: 28.3%
Learning Rate: 8.17824820842361e-06
Minibatch loss at step 3000: 5.500896
Minibatch accuracy: 26.6%
Validation accuracy: 34.7%
Learning Rate: 7.396261935355142e-06
Minibatch loss at step 4000: 5.214374
Minibatch accuracy: 34.4%
Validation accuracy: 39.9%
Learning Rate: 6.68904749545618e-06


KeyboardInterrupt: 

In [ ]:
best 94.7%

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---
